# Import Required Libraries
Import the necessary libraries, including pandas, plotly, and dash.

In [2]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc

# Load and Inspect Data
Load the events.csv file and inspect the data to understand its structure.

In [ ]:
# Load and Inspect Data

# Load the events.csv file into a DataFrame
events_df = pd.read_csv('events.csv')

# Display the first few rows of the DataFrame to inspect the data
events_df.head()

# Display the summary statistics of the DataFrame
events_df.describe()

# Display the information about the DataFrame including data types and non-null counts
events_df.info()

# Preprocess Data
Preprocess the data to convert time columns to datetime objects and filter data for the selected day.

In [ ]:
# Convert 'start_time' and 'end_time' columns to datetime objects
events_df['start_time'] = pd.to_datetime(events_df['start_time'])
events_df['end_time'] = pd.to_datetime(events_df['end_time'])

# Function to filter data for the selected day
def filter_data_for_day(df, selected_date):
    # Convert selected_date to datetime
    selected_date = pd.to_datetime(selected_date)
    # Filter the DataFrame for the selected day
    filtered_df = df[(df['start_time'].dt.date == selected_date.date()) | (df['end_time'].dt.date == selected_date.date())]
    return filtered_df

# Example usage: Filter data for a specific date (e.g., '2023-10-01')
selected_date = '2023-10-01'
filtered_events_df = filter_data_for_day(events_df, selected_date)

# Display the filtered DataFrame
filtered_events_df.head()

# Create Date Picker
Create a date picker component using Dash to allow users to select a date.

In [ ]:
# Create Date Picker

# Create a Dash app instance
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the layout of the app
app.layout = html.Div([
    # Date Picker component
    dcc.DatePickerSingle(
        id='date-picker',
        date=pd.to_datetime('today').date(),  # Default to today's date
        display_format='YYYY-MM-DD'
    ),
    # Div to display the selected date
    html.Div(id='output-container-date-picker')
])

# Callback to update the output based on the selected date
@app.callback(
    Output('output-container-date-picker', 'children'),
    Input('date-picker', 'date')
)
def update_output(date):
    return f'Selected Date: {date}'

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

# Generate Gantt Chart
Use Plotly to generate a Gantt chart that shows room availability, grouping rooms by buildings and floors.

In [3]:
# Generate Gantt Chart

import plotly.express as px

# Function to generate Gantt chart
def generate_gantt_chart(df):
    fig = px.timeline(
        df,
        x_start="start_time",
        x_end="end_time",
        y="room",
        color="name",
        title="Room Availability",
        labels={"name": "Event Name"},
        category_orders={"room": sorted(df["room"].unique())}
    )
    fig.update_yaxes(categoryorder="total ascending")
    fig.update_layout(
        xaxis_title="Time",
        yaxis_title="Room",
        showlegend=True
    )
    return fig

# Example usage: Generate Gantt chart for the filtered events DataFrame
gantt_chart = generate_gantt_chart(filtered_events_df)
gantt_chart.show()

NameError: name 'filtered_events_df' is not defined

# Display Gantt Chart
Display the Gantt chart on the Dash app.

In [ ]:
# Display Gantt Chart

# Update the layout of the app to include the Gantt chart
app.layout = html.Div([
    # Date Picker component
    dcc.DatePickerSingle(
        id='date-picker',
        date=pd.to_datetime('today').date(),  # Default to today's date
        display_format='YYYY-MM-DD'
    ),
    # Div to display the selected date
    html.Div(id='output-container-date-picker'),
    # Div to display the Gantt chart
    dcc.Graph(id='gantt-chart')
])

# Callback to update the Gantt chart based on the selected date
@app.callback(
    Output('gantt-chart', 'figure'),
    Input('date-picker', 'date')
)
def update_gantt_chart(date):
    # Filter data for the selected date
    filtered_df = filter_data_for_day(events_df, date)
    # Generate Gantt chart
    gantt_chart = generate_gantt_chart(filtered_df)
    return gantt_chart

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [4]:
import json

with open("data/events.json", "r", encoding='utf-8') as f:
    data = json.load(f)

len(data)

2399

In [22]:
df = pd.DataFrame(data)
df['building'] = df['audience_number'].apply(lambda x: x.split()[1])
df.head()

,name,description,pair,owner_user_wallet,audience_number,building
0,Теория функций комплексного переменного(Горяйн...,Теория функций комплексного переменного(Горяйн...,"{'name': 'Пара 1 1', 'time_slot_index': 1, 'we...",mipt,250 None,None
1,Основы цифровой обработки сигналов/ст.преподав...,Основы цифровой обработки сигналов/ст.преподав...,"{'name': 'Пара 1 3', 'time_slot_index': 3, 'we...",mipt,250 None,None
2,"Безопасность жизнедеятельности / ФПМИ(ФУПМ), И...","Безопасность жизнедеятельности / ФПМИ(ФУПМ), И...","{'name': 'Пара 1 6', 'time_slot_index': 6, 'we...",mipt,250 None,None
3,дополнительным главам общей физике ЛФИ Колдуно...,дополнительным главам общей физике ЛФИ Колдуно...,"{'name': 'Пара 1 7', 'time_slot_index': 7, 'we...",mipt,250 None,None
4,Кратные интегралы и теория поля /профессор Кар...,Кратные интегралы и теория поля /профессор Кар...,"{'name': 'Пара 2 1', 'time_slot_index': 1, 'we...",mipt,250 None,None


In [23]:
df["pair"].apply(lambda d: d['name']).unique()

array(['Пара 1 1', 'Пара 1 3', 'Пара 1 6', 'Пара 1 7', 'Пара 2 1',
       'Пара 2 2', 'Пара 2 3', 'Пара 2 4', 'Пара 2 7', 'Пара 2 8',
       'Пара 3 2', 'Пара 3 3', 'Пара 3 4', 'Пара 3 5', 'Пара 3 7',
       'Пара 4 1', 'Пара 4 3', 'Пара 4 5', 'Пара 4 8', 'Пара 4 6',
       'Пара 4 7', 'Пара 5 1', 'Пара 5 2', 'Пара 6 5', 'Пара 5 3',
       'Пара 5 4', 'Пара 5 5', 'Пара 5 7', 'Пара 6 1', 'Пара 6 2',
       'Пара 6 3', 'Пара 6 4', 'Пара 6 7', 'Пара 1 2', 'Пара 1 4',
       'Пара 2 5', 'Пара 2 6', 'Пара 3 6', 'Пара 3 8', 'Пара 4 2',
       'Пара 4 4', 'Пара 5 6', 'Пара 5 8', 'Пара 6 6', 'Пара 7 1',
       'Пара 7 2', 'Пара 7 3', 'Пара 7 4', 'Пара 7 5', 'Пара 7 6',
       'Пара 3 1', 'Пара 6 8', 'Пара 1 8', 'Пара 7 7', 'Пара 7 8',
       'Пара 1 5'], dtype=object)

In [24]:
df[df["pair"].apply(lambda d: d['name'] == 'Пара 1 6')]['pair'].apply(lambda d: d['description']).unique()

array(["Пара номер: Понедельник (6, '17:05', '18:30')"], dtype=object)

In [25]:
df['pair'].apply(lambda d: d['description']).unique()

array(["Пара номер: Понедельник (1, '09:00', '10:25')",
       "Пара номер: Понедельник (3, '12:10', '13:55')",
       "Пара номер: Понедельник (6, '17:05', '18:30')",
       "Пара номер: Понедельник (7, '18:30', '20:00')",
       "Пара номер: Вторник (1, '09:00', '10:25')",
       "Пара номер: Вторник (2, '10:25', '12:10')",
       "Пара номер: Вторник (3, '12:10', '13:55')",
       "Пара номер: Вторник (4, '13:55', '15:30')",
       "Пара номер: Вторник (7, '18:30', '20:00')",
       "Пара номер: Вторник (8, '20:00', '22:00')",
       "Пара номер: Среда (2, '10:25', '12:10')",
       "Пара номер: Среда (3, '12:10', '13:55')",
       "Пара номер: Среда (4, '13:55', '15:30')",
       "Пара номер: Среда (5, '15:30', '17:05')",
       "Пара номер: Среда (7, '18:30', '20:00')",
       "Пара номер: Четверг (1, '09:00', '10:25')",
       "Пара номер: Четверг (3, '12:10', '13:55')",
       "Пара номер: Четверг (5, '15:30', '17:05')",
       "Пара номер: Четверг (8, '20:00', '22:00')",
      

In [26]:
df[['day', 'time']] = df['pair'].apply(lambda d: pd.Series(d['name'].split()[1:]))
df

,name,description,pair,owner_user_wallet,audience_number,building,day,time
0,Теория функций комплексного переменного(Горяйн...,Теория функций комплексного переменного(Горяйн...,"{'name': 'Пара 1 1', 'time_slot_index': 1, 'we...",mipt,250 None,None,1,1
1,Основы цифровой обработки сигналов/ст.преподав...,Основы цифровой обработки сигналов/ст.преподав...,"{'name': 'Пара 1 3', 'time_slot_index': 3, 'we...",mipt,250 None,None,1,3
2,"Безопасность жизнедеятельности / ФПМИ(ФУПМ), И...","Безопасность жизнедеятельности / ФПМИ(ФУПМ), И...","{'name': 'Пара 1 6', 'time_slot_index': 6, 'we...",mipt,250 None,None,1,6
3,дополнительным главам общей физике ЛФИ Колдуно...,дополнительным главам общей физике ЛФИ Колдуно...,"{'name': 'Пара 1 7', 'time_slot_index': 7, 'we...",mipt,250 None,None,1,7
4,Кратные интегралы и теория поля /профессор Кар...,Кратные интегралы и теория поля /профессор Кар...,"{'name': 'Пара 2 1', 'time_slot_index': 1, 'we...",mipt,250 None,None,2,1
...,...,...,...,...,...,...,...,...
2394,4к иностр тополог Елишев,4к иностр тополог Елишев,"{'name': 'Пара 5 2', 'time_slot_index': 2, 'we...",mipt,202 УПМ,УПМ,5,2
2395,4к иностр тополог Елишев,4к иностр тополог Елишев,"{'name': 'Пара 5 3', 'time_slot_index': 3, 'we...",mipt,202 УПМ,УПМ,5,3
2396,Общая физика 5 к. Рыбакова А.К.,Общая физика 5 к. Рыбакова А.К.,"{'name': 'Пара 5 4', 'time_slot_index': 4, 'we...",mipt,202 УПМ,УПМ,5,4
2397,1к иностр физика Мусин С,1к иностр физика Мусин С,"{'name': 'Пара 5 6', 'time_slot_index': 6, 'we...",mipt,202 УПМ,УПМ,5,6


In [27]:
WEEKDAYS = {
    "1": "Monday",
    "2": "Tuesday",
    "3": "Wednesday",
    "4": "Thursday",
    "5": "Friday",
    "6": "Saturday",
    "7": "Sunday",
}

TIME_SLOTS = {
    "1": ("09:00", "10:25"),
    "2": ("10:45", "12:10"),
    "3": ("12:20", "13:45"),
    "4": ("13:55", "15:20"),
    "5": ("15:30", "16:55"),
    "6": ("17:05", "18:30"),
    "7": ("18:35", "20:00"),
    "8": ("20:00", "22:00"),
}

In [28]:
df[['time_start', 'time_finish']] = df['time'].apply(lambda x: pd.Series(TIME_SLOTS[x]))
df['weekday'] = df['day'].map(WEEKDAYS)
df

,name,description,pair,owner_user_wallet,audience_number,building,day,time,time_start,time_finish,weekday
0,Теория функций комплексного переменного(Горяйн...,Теория функций комплексного переменного(Горяйн...,"{'name': 'Пара 1 1', 'time_slot_index': 1, 'we...",mipt,250 None,None,1,1,09:00,10:25,Monday
1,Основы цифровой обработки сигналов/ст.преподав...,Основы цифровой обработки сигналов/ст.преподав...,"{'name': 'Пара 1 3', 'time_slot_index': 3, 'we...",mipt,250 None,None,1,3,12:20,13:45,Monday
2,"Безопасность жизнедеятельности / ФПМИ(ФУПМ), И...","Безопасность жизнедеятельности / ФПМИ(ФУПМ), И...","{'name': 'Пара 1 6', 'time_slot_index': 6, 'we...",mipt,250 None,None,1,6,17:05,18:30,Monday
3,дополнительным главам общей физике ЛФИ Колдуно...,дополнительным главам общей физике ЛФИ Колдуно...,"{'name': 'Пара 1 7', 'time_slot_index': 7, 'we...",mipt,250 None,None,1,7,18:35,20:00,Monday
4,Кратные интегралы и теория поля /профессор Кар...,Кратные интегралы и теория поля /профессор Кар...,"{'name': 'Пара 2 1', 'time_slot_index': 1, 'we...",mipt,250 None,None,2,1,09:00,10:25,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...
2394,4к иностр тополог Елишев,4к иностр тополог Елишев,"{'name': 'Пара 5 2', 'time_slot_index': 2, 'we...",mipt,202 УПМ,УПМ,5,2,10:45,12:10,Friday
2395,4к иностр тополог Елишев,4к иностр тополог Елишев,"{'name': 'Пара 5 3', 'time_slot_index': 3, 'we...",mipt,202 УПМ,УПМ,5,3,12:20,13:45,Friday
2396,Общая физика 5 к. Рыбакова А.К.,Общая физика 5 к. Рыбакова А.К.,"{'name': 'Пара 5 4', 'time_slot_index': 4, 'we...",mipt,202 УПМ,УПМ,5,4,13:55,15:20,Friday
2397,1к иностр физика Мусин С,1к иностр физика Мусин С,"{'name': 'Пара 5 6', 'time_slot_index': 6, 'we...",mipt,202 УПМ,УПМ,5,6,17:05,18:30,Friday


In [29]:
off = df[df['name'] != df['description']].sample(1)
off

,name,description,pair,owner_user_wallet,audience_number,building,day,time,time_start,time_finish,weekday
729,А.К. 3: 2 из 4: Эффективные алгоритмы/ ст.преп...,А.К. 3: 2 из 4: Эффективные алгоритмы/ ст.преп...,"{'name': 'Пара 5 7', 'time_slot_index': 7, 'we...",mipt,5.18 Цифра,Цифра,5,7,18:35,20:00,Friday


In [30]:
import datetime as dtt

In [31]:
selected_date = dtt.datetime.strptime("2024-11-18", '%Y-%m-%d')
weekday = str(selected_date.weekday()+1)

In [32]:
filtered_df = df[(df['day'] == weekday)]
filtered_df

,name,description,pair,owner_user_wallet,audience_number,building,day,time,time_start,time_finish,weekday
0,Теория функций комплексного переменного(Горяйн...,Теория функций комплексного переменного(Горяйн...,"{'name': 'Пара 1 1', 'time_slot_index': 1, 'we...",mipt,250 None,None,1,1,09:00,10:25,Monday
1,Основы цифровой обработки сигналов/ст.преподав...,Основы цифровой обработки сигналов/ст.преподав...,"{'name': 'Пара 1 3', 'time_slot_index': 3, 'we...",mipt,250 None,None,1,3,12:20,13:45,Monday
2,"Безопасность жизнедеятельности / ФПМИ(ФУПМ), И...","Безопасность жизнедеятельности / ФПМИ(ФУПМ), И...","{'name': 'Пара 1 6', 'time_slot_index': 6, 'we...",mipt,250 None,None,1,6,17:05,18:30,Monday
3,дополнительным главам общей физике ЛФИ Колдуно...,дополнительным главам общей физике ЛФИ Колдуно...,"{'name': 'Пара 1 7', 'time_slot_index': 7, 'we...",mipt,250 None,None,1,7,18:35,20:00,Monday
34,Введение в математический анализ / профессор З...,Введение в математический анализ / профессор З...,"{'name': 'Пара 1 2', 'time_slot_index': 2, 'we...",mipt,300 None,None,1,2,10:45,12:10,Monday
...,...,...,...,...,...,...,...,...,...,...,...
2328,ЛФИ факультатив. Основы моделирования физическ...,ЛФИ факультатив. Основы моделирования физическ...,"{'name': 'Пара 1 3', 'time_slot_index': 3, 'we...",mipt,203 УПМ,УПМ,1,3,12:20,13:45,Monday
2351,"6 к иностр Введение в физику плазмы, Введение ...","6 к иностр Введение в физику плазмы, Введение ...","{'name': 'Пара 1 1', 'time_slot_index': 1, 'we...",mipt,204 УПМ,УПМ,1,1,09:00,10:25,Monday
2352,"6 к иностр Введение в физику плазмы, Введение ...","6 к иностр Введение в физику плазмы, Введение ...","{'name': 'Пара 1 2', 'time_slot_index': 2, 'we...",mipt,204 УПМ,УПМ,1,2,10:45,12:10,Monday
2353,"6 к иностр Введение в физику плазмы, Введение ...","6 к иностр Введение в физику плазмы, Введение ...","{'name': 'Пара 1 3', 'time_slot_index': 3, 'we...",mipt,204 УПМ,УПМ,1,3,12:20,13:45,Monday


In [33]:
sorted(df["room"].unique())

KeyError: 'room'

In [34]:
df['building'].unique()

array(['None', 'УПМ', 'Квант', 'Цифра', 'КПМ', 'ГК', 'БК'], dtype=object)